In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv("Combined_Flights_2022.csv")
df.head(5)

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",GJT,DEN,False,False,1133,1123.0,0.0,-10.0,...,1140.0,1220.0,8.0,1245,-17.0,0.0,-2.0,1200-1259,1,0
1,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",HRL,IAH,False,False,732,728.0,0.0,-4.0,...,744.0,839.0,9.0,849,-1.0,0.0,-1.0,0800-0859,2,0
2,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1529,1514.0,0.0,-15.0,...,1535.0,1622.0,14.0,1639,-3.0,0.0,-1.0,1600-1659,2,0
3,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,GPT,False,False,1435,1430.0,0.0,-5.0,...,1446.0,1543.0,4.0,1605,-18.0,0.0,-2.0,1600-1659,2,0
4,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1135,1135.0,0.0,0.0,...,1154.0,1243.0,8.0,1245,6.0,0.0,0.0,1200-1259,2,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4078318 entries, 0 to 4078317
Data columns (total 61 columns):
 #   Column                                   Dtype  
---  ------                                   -----  
 0   FlightDate                               object 
 1   Airline                                  object 
 2   Origin                                   object 
 3   Dest                                     object 
 4   Cancelled                                bool   
 5   Diverted                                 bool   
 6   CRSDepTime                               int64  
 7   DepTime                                  float64
 8   DepDelayMinutes                          float64
 9   DepDelay                                 float64
 10  ArrTime                                  float64
 11  ArrDelayMinutes                          float64
 12  AirTime                                  float64
 13  CRSElapsedTime                           float64
 14  ActualElapsedTime 

In [4]:
df.isnull().sum().sort_values(ascending=False).head(30)

ArrDelayMinutes                133402
AirTime                        133402
ActualElapsedTime              133402
ArrDel15                       133402
ArrivalDelayGroups             133402
ArrDelay                       133402
WheelsOn                       124242
TaxiIn                         124242
ArrTime                        124239
TaxiOut                        122666
WheelsOff                      122666
DepDelay                       120495
DepDelayMinutes                120495
DepartureDelayGroups           120495
DepDel15                       120495
DepTime                        120433
Tail_Number                     26795
Quarter                             0
Month                               0
Distance                            0
Year                                0
CRSDepTime                          0
CRSElapsedTime                      0
Origin                              0
Cancelled                           0
FlightDate                          0
Dest        

In [5]:
num_rows_with_na = df.isnull().any(axis=1).sum()
print(num_rows_with_na)

133402


In [6]:
object_cols = df.select_dtypes(include='object').columns
print(object_cols)

Index(['FlightDate', 'Airline', 'Origin', 'Dest', 'Marketing_Airline_Network',
       'Operated_or_Branded_Code_Share_Partners',
       'IATA_Code_Marketing_Airline', 'Operating_Airline',
       'IATA_Code_Operating_Airline', 'Tail_Number', 'OriginCityName',
       'OriginState', 'OriginStateName', 'DestCityName', 'DestState',
       'DestStateName', 'DepTimeBlk', 'ArrTimeBlk'],
      dtype='object')


In [7]:
int_cols = df.select_dtypes(include='integer').columns
print(int_cols)

Index(['CRSDepTime', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'DOT_ID_Marketing_Airline', 'Flight_Number_Marketing_Airline',
       'DOT_ID_Operating_Airline', 'Flight_Number_Operating_Airline',
       'OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID',
       'OriginStateFips', 'OriginWac', 'DestAirportID', 'DestAirportSeqID',
       'DestCityMarketID', 'DestStateFips', 'DestWac', 'CRSArrTime',
       'DistanceGroup', 'DivAirportLandings'],
      dtype='object')


In [8]:
float_cols = df.select_dtypes(include='float').columns
print(float_cols)

Index(['DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime', 'ArrDelayMinutes',
       'AirTime', 'CRSElapsedTime', 'ActualElapsedTime', 'Distance',
       'DepDel15', 'DepartureDelayGroups', 'TaxiOut', 'WheelsOff', 'WheelsOn',
       'TaxiIn', 'ArrDelay', 'ArrDel15', 'ArrivalDelayGroups'],
      dtype='object')


In [11]:
# Identify boolean columns and convert them to int (True -> 1, False -> 0)
bool_cols = df.select_dtypes(include=['bool']).columns
df[bool_cols] = df[bool_cols].astype(int)

# Check that 'cancelled' column exists and is numeric or boolean
if 'Cancelled' not in df.columns:
    raise ValueError("Column 'cancelled' not found in the dataset.")

# Convert 'cancelled' to int if boolean
if df['Cancelled'].dtype == 'bool':
    df['Cancelled'] = df['Cancelled'].astype(int)

# Identify categorical columns
cat_cols = df.select_dtypes(include=['object', 'category']).columns

# One-hot encode categorical variables (except 'cancelled')
cat_cols = [col for col in cat_cols if col != 'Cancelled']
df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)

# Compute correlation matrix
corr_matrix = df_encoded.corr()

# Extract correlation with 'cancelled'
cancelled_corr = corr_matrix['Cancelled'].drop('Cancelled')  # exclude self-correlation

# Rank features by absolute correlation (descending)
ranked_corr = cancelled_corr.abs().sort_values(ascending=False)

print("Features ranked by absolute correlation to 'cancelled':\n")
print(ranked_corr)

# Optional: show actual signed correlation values as well
print("\nFeatures with correlation sign:\n")
print(cancelled_corr[ranked_corr.index])

Features ranked by absolute correlation to 'cancelled':

Tail_Number                                0.075068
FlightDate                                 0.069587
Month                                      0.065174
DivAirportLandings                         0.053723
Quarter                                    0.051262
OriginWac                                  0.042314
DestWac                                    0.041696
DayofMonth                                 0.040252
DepartureDelayGroups                       0.032496
DayOfWeek                                  0.030750
DOT_ID_Operating_Airline                   0.029923
DepDelay                                   0.026592
DepDelayMinutes                            0.026395
OriginStateFips                            0.021884
DOT_ID_Marketing_Airline                   0.021865
DepDel15                                   0.021788
OriginStateName                            0.021741
OriginState                                0.021386
DestSta

In [12]:
# Identify boolean columns and convert them to int (True -> 1, False -> 0)
bool_cols = df.select_dtypes(include=['bool']).columns
df[bool_cols] = df[bool_cols].astype(int)

# Check that 'cancelled' column exists and is numeric or boolean
if 'Cancelled' not in df.columns:
    raise ValueError("Column 'cancelled' not found in the dataset.")

# Convert 'cancelled' to int if boolean
if df['Cancelled'].dtype == 'bool':
    df['Cancelled'] = df['Cancelled'].astype(int)

# Identify categorical columns
cat_cols = df.select_dtypes(include=['object', 'category']).columns

# One-hot encode categorical variables (except 'cancelled')
cat_cols = [col for col in cat_cols if col != 'Cancelled']
df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)

# Compute correlation matrix
corr_matrix = df_encoded.corr()

# Extract correlation with 'cancelled'
cancelled_corr = corr_matrix['ArrDel15'].drop('ArrDel15')  # exclude self-correlation

# Rank features by absolute correlation (descending)
ranked_corr = cancelled_corr.abs().sort_values(ascending=False)

print("Features ranked by absolute correlation to 'ArrDel15':\n")
print(ranked_corr)

# Optional: show actual signed correlation values as well
print("\nFeatures with correlation sign:\n")
print(cancelled_corr[ranked_corr.index])

Features ranked by absolute correlation to 'ArrDel15':

ArrivalDelayGroups                         7.601004e-01
DepDel15                                   7.455516e-01
DepartureDelayGroups                       6.832345e-01
ArrDelay                                   5.762299e-01
ArrDelayMinutes                            5.289249e-01
DepDelay                                   5.106031e-01
DepDelayMinutes                            4.949143e-01
TaxiOut                                    2.348745e-01
DepTime                                    2.002401e-01
WheelsOff                                  1.968246e-01
DepTimeBlk                                 1.596796e-01
CRSDepTime                                 1.589376e-01
ArrTimeBlk                                 1.388825e-01
TaxiIn                                     1.387688e-01
CRSArrTime                                 1.272109e-01
ActualElapsedTime                          8.643988e-02
WheelsOn                                   8.567